In [ ]:
# Py Spark using MLib on Classificatio of flower using sizes

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 5.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 17.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425347 sha256=15f8b0d5daa82ebcd9140c4a7359ab05368c8e7306cc18f047aec8aa6d061cb4
  Stored in directory: /Users/manoj/Library/Caches/pip/wheels/38/df/61/8c121f50c3cffd77f8178180dd232d90b3b99d1bd61fb6d6be
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
spark = SparkSession.builder.appName('mllib').getOrCreate()

In [5]:
spark

24/02/25 13:03:48 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
df = spark.read.csv("/Users/manoj/Desktop/Spark/bezdekIris_data.csv", inferSchema=True).toDF("sep_len", "sep_wid", "pet_len", "pet_wid", "label")

In [10]:
df.show()

+-------+-------+-------+-------+-----------+
|sep_len|sep_wid|pet_len|pet_wid|      label|
+-------+-------+-------+-------+-----------+
|    5.1|    3.5|    1.4|    0.2|Iris-setosa|
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|
|    5.4|    3.9|    1.7|    0.4|Iris-setosa|
|    4.6|    3.4|    1.4|    0.3|Iris-setosa|
|    5.0|    3.4|    1.5|    0.2|Iris-setosa|
|    4.4|    2.9|    1.4|    0.2|Iris-setosa|
|    4.9|    3.1|    1.5|    0.1|Iris-setosa|
|    5.4|    3.7|    1.5|    0.2|Iris-setosa|
|    4.8|    3.4|    1.6|    0.2|Iris-setosa|
|    4.8|    3.0|    1.4|    0.1|Iris-setosa|
|    4.3|    3.0|    1.1|    0.1|Iris-setosa|
|    5.8|    4.0|    1.2|    0.2|Iris-setosa|
|    5.7|    4.4|    1.5|    0.4|Iris-setosa|
|    5.4|    3.9|    1.3|    0.4|Iris-setosa|
|    5.1|    3.5|    1.4|    0.3|Iris-setosa|
|    5.7|    3.8|    1.7|    0.3|I

In [11]:
df.printSchema()

root
 |-- sep_len: double (nullable = true)
 |-- sep_wid: double (nullable = true)
 |-- pet_len: double (nullable = true)
 |-- pet_wid: double (nullable = true)
 |-- label: string (nullable = true)



In [12]:
from pyspark.sql.functions import mean
df.select(*[mean(c) for c in df.columns]).show()

+-----------------+-----------------+------------------+-----------------+----------+
|     avg(sep_len)|     avg(sep_wid)|      avg(pet_len)|     avg(pet_wid)|avg(label)|
+-----------------+-----------------+------------------+-----------------+----------+
|5.843333333333335|3.057333333333334|3.7580000000000027|1.199333333333334|      NULL|
+-----------------+-----------------+------------------+-----------------+----------+



In [13]:
vector_assembler = VectorAssembler(inputCols=["sep_len", "sep_wid", "pet_len", "pet_wid"], outputCol="features")
df_temp = vector_assembler.transform(df)
df_temp.show(3)

+-------+-------+-------+-------+-----------+-----------------+
|sep_len|sep_wid|pet_len|pet_wid|      label|         features|
+-------+-------+-------+-------+-----------+-----------------+
|    5.1|    3.5|    1.4|    0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
+-------+-------+-------+-------+-----------+-----------------+
only showing top 3 rows



In [18]:
df = df_temp.drop('sep_len', 'sep_wid', 'pet_len', 'pet_wid')
df.show(3)

+-----------+-----------------+
|      label|         features|
+-----------+-----------------+
|Iris-setosa|[5.1,3.5,1.4,0.2]|
|Iris-setosa|[4.9,3.0,1.4,0.2]|
|Iris-setosa|[4.7,3.2,1.3,0.2]|
+-----------+-----------------+
only showing top 3 rows



In [19]:
l_indexer = StringIndexer(inputCol="label", outputCol="labelIndex" )
df = l_indexer.fit(df).transform(df)


In [20]:
df.show(3)

+-----------+-----------------+----------+
|      label|         features|labelIndex|
+-----------+-----------------+----------+
|Iris-setosa|[5.1,3.5,1.4,0.2]|       0.0|
|Iris-setosa|[4.9,3.0,1.4,0.2]|       0.0|
|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
+-----------+-----------------+----------+
only showing top 3 rows



In [ ]:
# Decision Tree 

In [21]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

In [22]:
dt = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="features")
model = dt.fit(trainingData)

In [23]:
predictions = model.transform(testData)

In [24]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="labelIndex", predictionCol="prediction",\
metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy: ",accuracy)
print("Test Error = %g " % (1.0 - accuracy))

Accuracy:  0.9555555555555556
Test Error = 0.0444444 


In [ ]:
# Naive Bias Classifier

In [25]:
splits = df.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

In [26]:
nb = NaiveBayes(labelCol="labelIndex",\
featuresCol="features", smoothing=1.0,\
modelType="multinomial")
model = nb.fit(train)

In [27]:
predictions = model.transform(test)
predictions.select("label", "labelIndex",
"probability", "prediction").show()

+-----------+----------+--------------------+----------+
|      label|labelIndex|         probability|prediction|
+-----------+----------+--------------------+----------+
|Iris-setosa|       0.0|[0.73502886090153...|       0.0|
|Iris-setosa|       0.0|[0.64985557718392...|       0.0|
|Iris-setosa|       0.0|[0.55508830217120...|       0.0|
|Iris-setosa|       0.0|[0.71765512033989...|       0.0|
|Iris-setosa|       0.0|[0.66081341393231...|       0.0|
|Iris-setosa|       0.0|[0.64974667051749...|       0.0|
|Iris-setosa|       0.0|[0.74441220910135...|       0.0|
|Iris-setosa|       0.0|[0.64883669266108...|       0.0|
|Iris-setosa|       0.0|[0.73798193287052...|       0.0|
|Iris-setosa|       0.0|[0.59910017548780...|       0.0|
|Iris-setosa|       0.0|[0.76033164161201...|       0.0|
|Iris-setosa|       0.0|[0.72728619556167...|       0.0|
|Iris-setosa|       0.0|[0.72713552086094...|       0.0|
|Iris-setosa|       0.0|[0.70836433322158...|       0.0|
|Iris-setosa|       0.0|[0.7468

24/02/25 13:49:35 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/02/25 13:49:35 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [28]:
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex",\
predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy:", str(accuracy))

Test set accuracy: 0.9682539682539683
